<h1  align="center"><b> CÁLCULO DA MÉDIA MÓVEL </b></h1>

`Objetivo Geral:` Calcular a média móvel de novos casos e obtos por Covid em São José dos Campos.

`Dados:` Os dados foram retirados do repositório [dados-covid-sp](https://github.com/seade-R/dados-covid-sp) da Fundação Seade no dia 19/12/2023.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

<h2 align="center"><b> Análise Inicial </b></h2>

`Objetivo da Etapa:` Importar os dados e verificar se há algum problema.

In [8]:
tabela_covid = pd.read_csv("Dados Covid SP.csv", sep=";") # Importando os dados
tabela_covid = tabela_covid.loc[tabela_covid['nome_munic'] == 'São José dos Campos'] # Selecionando apenas os dados de São José dos Campos
tabela_covid = tabela_covid.drop(['nome_munic', 'codigo_ibge', 'pop'], axis=1) # Removendo colunas desnecessárias

In [9]:
tabela_covid.dtypes # Verificando os tipos de dados de cada coluna

datahora         object
casos             int64
casos_novos       int64
casos_mm7d       object
obitos            int64
obitos_novos      int64
obitos_mm7d      object
semana_epidem     int64
dtype: object

`Tipos de Atributos:`
<br>🔹 object -> strings
<br>🔹 int64: -> inteiros
<br>🔹 float64 -> reais
<br>🔹 complex -> complexos

&emsp;&emsp; As colunas casos_mm7d e obitos_mm7d deveriam ser numéricas, mas estão sendo reconhecidas como strings. Isso acontece porque esses valores estão utilizando vírgula como separador decimal. Para resolver esse problema, vamos substituir a vírgula por ponto e converter para um valor numérico.

In [10]:
tabela_covid['casos_mm7d'] = tabela_covid['casos_mm7d'].str.replace(',', '.') # Substituindo vírgula por ponto
tabela_covid['casos_mm7d'] = pd.to_numeric(tabela_covid['casos_mm7d']) # Transformando a coluna casos_mm7d em float

tabela_covid['obitos_mm7d'] = tabela_covid['obitos_mm7d'].str.replace(',', '.') # Substituindo vírgula por ponto
tabela_covid['obitos_mm7d'] = pd.to_numeric(tabela_covid['obitos_mm7d']) # Transformando a coluna obitos_mm7d em float

In [11]:
tabela_covid.dtypes

datahora          object
casos              int64
casos_novos        int64
casos_mm7d       float64
obitos             int64
obitos_novos       int64
obitos_mm7d      float64
semana_epidem      int64
dtype: object

In [12]:
tabela_covid.isnull().sum() # Verificando se há valores nulos

datahora         0
casos            0
casos_novos      0
casos_mm7d       0
obitos           0
obitos_novos     0
obitos_mm7d      0
semana_epidem    0
dtype: int64

&emsp;&emsp; Não há dados faltantes, mas há diversas linhas com apenas zeros. Vamos excluir parte dessas linhas (deixando apenas as que ocorrem na semana anterior ao primeiro caso).

In [13]:
ultima_ocorrência = tabela_covid.loc[tabela_covid['casos'] == 0]['semana_epidem'].max()
tabela_covid = tabela_covid.loc[tabela_covid['semana_epidem'] >= ultima_ocorrência - 1]
tabela_covid.reset_index(drop=True, inplace=True) # Resetando o índice

<h2 align="center"><b> Média Móvel de Novos Casos </b></h2>

`Objetivo da Etapa:` Calcular a média móvel de novos casos por Covid em São José dos Campos e plotar o gráfico.

In [ ]:
série_casos = pd.Series(tabela_covid.casos_novos.values, index = tabela_covid.datahora) # Criando uma série com os casos novos
série_casos.plot() # Plotando a série

![série_caos.plot()](./Gráficos/série_casos.png)

In [ ]:
média_casos = série_casos.rolling(window = 7).mean() # Calculando a média móvel

plt.title('Casos Novos')
plt.plot(série_casos, label = 'Série Real', color = 'blue')
plt.plot(média_casos, label = 'Média Móvel Simples', color = 'red')
plt.legend(loc = 'best')
plt.show()

![média_casos](./Gráficos/média_casos.png)

In [ ]:
média_casos_centralizada = série_casos.rolling(window = 7, center = True).mean() # Calculando a média móvel centralizada

plt.title('Casos Novos')
plt.plot(série_casos, label = 'Série Real', color = 'blue')
plt.plot(média_casos_centralizada, label = 'Média Móvel Centralizada', color = 'red')
plt.legend(loc = 'best')
plt.show()

![média_casos_centralizada](./Gráficos/média_casos_centralizada.png)

<h2 align="center"><b> Média Móvel de Novos Óbitos </b></h2>

`Objetivo da Etapa:` Calcular a média móvel de novos óbitos por Covid em São José dos Campos e plotar o gráfico.

In [ ]:
série_óbitos = pd.Series(tabela_covid.obitos_novos.values, index = tabela_covid.datahora) # Criando uma série com os óbitos novos
série_óbitos.plot() # Plotando a série

![série_óbitos.plot()](./Gráficos/série_óbitos.png)

In [ ]:
média_óbitos = série_óbitos.rolling(window = 7).mean() # Calculando a média móvel

plt.title('Óbitos Novos')
plt.plot(série_óbitos, label = 'Série Real', color = 'blue')
plt.plot(média_óbitos, label = 'Média Simples', color = 'red')
plt.legend(loc = 'best')
plt.show()

![média_óbitos](./Gráficos/média_óbitos.png)

In [ ]:
média_óbitos_centralizada = série_óbitos.rolling(window = 7, center = True).mean() # Calculando a média móvel centralizada

plt.title('Óbitos Novos')
plt.plot(série_óbitos, label = 'Série Real', color = 'blue')
plt.plot(média_óbitos_centralizada, label = 'Média Centralizada', color = 'red')
plt.legend(loc = 'best')
plt.show()

![média_óbitos](./Gráficos/média_óbitos_centralizada.png)